In [1]:
run load_libs.py

<IPython.core.display.Javascript object>

Pinball loss is commonly used in [quantile
regression](https://en.wikipedia.org/wiki/Quantile_regression).

Definition:


\begin{align}
\mathbb{E}[L_{\tau}]
&= \iint (t - y(\mathbf{x})) (\tau - \mathbb{I}_{t < y(\mathbf{x})}) p(\mathbf{x}, t) d\mathbf{x} dt
\end{align}

To minimize it, we first calculate the derviative wrt. $y(\mathbf{x})$,

\begin{align}
\frac{\partial \mathbb{E}[L_{\tau}]}{\partial y(\mathbf{x})}
&= - \int (\tau - \mathbb{I}_{t < y(\mathbf{x})}) p(\mathbf{x}, t) dt \\
&= - \int_{-\infty}^{y(\mathbf{x})} (\tau - 1) p(\mathbf{x}, t) dt - \int_{y(\mathbf{x})}^{\infty} \tau p(\mathbf{x}, t) dt \\
&= - \int_{-\infty}^{y(\mathbf{x})} (\tau - 1) p(t|\mathbf{x}) p(\mathbf{x}) dt - \int_{y(\mathbf{x})}^{\infty} \tau p(t|\mathbf{x}) p(\mathbf{x})  dt \\
\end{align}

then set the derivative to zero,

\begin{align}
(\tau - 1) \int_{-\infty}^{y(\mathbf{x})} p(t|\mathbf{x}) d\mathbf{x} + \tau \int_{y(\mathbf{x})}^{\infty}  p(t|\mathbf{x}) d\mathbf{x}
&= 0 \\
(\tau - 1) F_{t|\mathbf{x}}(y(\mathbf{x}))
&= - \tau (1 - F_{t|\mathbf{x}}(y(\mathbf{x}))) \\
F_{t|\mathbf{x}}(y(\mathbf{x})) &= \tau \\
y(\mathbf{x}) &= F_{t|\mathbf{x}}^{-1}(\tau)
\end{align}

Note,

* $\tau \in [0, 1]$.
* $F$ represents a CDF.

So when minimizing the pinball loss, the model is also trying to predict the
conditional $\tau$-th quantile of $t$ given $\mathbf{x}$, i.e.
$F_{t|\mathbf{x}}^{-1}(\tau)$.

Note, when $\tau = 0.5$, pinball loss become equilvalent to MAE scaled by 2, and
both are trying to predict the conditional median.

# Visualization

In [2]:
def pinball_loss(y, t, tau):
    return (t - y) * (tau - (t < y).astype(int))

In [3]:
dfs = []
t = 0
ys = np.arange(-10, 11, 1)
for tau in [0, 0.25, 0.5, 0.75, 1]:
    dfs.append(
        pd.DataFrame(
            {
                "delta": ys,
                "pinball_loss": pinball_loss(ys, t, tau),
                "τ": tau,
            }
        )
    )
ndf = pd.concat(dfs)

alt.Chart(ndf).mark_line().encode(
    x=alt.X("delta",  title="Δ = y(x) - t"),
    y="pinball_loss",
    color="τ:O",
)

alt.Chart(...)

Visualize in separate panels

In [4]:
alt.Chart(ndf).mark_line().encode(
    x=alt.X("delta", title="Δ = y(x) - t"),
    y="pinball_loss",
).properties(width=120, height=60).facet(facet="τ:O", columns=11)

alt.FacetChart(...)

The following chart shows the angle between two slopes for different τ values.

Note,

* when $y(\mathbf{x}) > t$, the slope is $1 - \tau$,
* when $y(\mathbf{x}) < t$, the slope is $- \tau$.

So the angle between two slopes is $180 - [(180 - \arctan(-\tau)) + \arctan(1 - \tau) ]$.

In [5]:
taus = np.arange(0, 1.1, 0.1)

def radian2degree(r):
    return r / np.pi * 180


alt.Chart(
    pd.DataFrame(
        {
            "τ": taus,
            "degree": 180 - radian2degree(np.pi - np.arctan(-taus) + np.arctan(1 - taus)),
        }
    )
).mark_line(point=True).encode(x='τ', y=alt.Y('degree', scale=alt.Scale(zero=False)))

alt.Chart(...)

The key observation is that the angle between slopes doesn't change in a linear rate as $\tau$ increases.